In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostClassifier, Pool
from rdkit import Chem
from rdkit.Chem import Lipinski, Descriptors, MolSurf, AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from scipy import stats
import pickle
%matplotlib inline

In [ ]:
import pandas as pd

# Загружаем файл Excel
df = pd.read_csv('data/SEC_SYN_new.csv')

# Устанавливаем опцию для отображения всех колонок (если это необходимо для отладки, иначе можно убрать)
pd.set_option('display.max_columns', None)

# Удаляем строки, содержащие хотя бы одно пропущенное значение (NaN)
df = df.dropna()

# Сбрасываем индексы DataFrame после удаления строк, чтобы они были последовательными
# inplace=True изменяет DataFrame напрямую, drop=True предотвращает добавление старых индексов как новой колонки
df.reset_index(drop=True, inplace=True)

# Выводим названия колонок DataFrame
print(df.columns)

### Дескрипторы лиганда

In [ ]:
# Импортируем необходимые библиотеки
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
from tqdm import tqdm
import pandas as pd # Добавлен импорт pandas для примера

# Словарь с вашими лигандами
ligand_smiles = {
    'BTC': 'C1(=CC(=CC(=C1)C(=O)[O-])C(=O)[O-])C(=O)[O-]',
    'BDC': 'O=C([O-])C1=CC=C(C=C1)C(=O)[O-]',
    'NH2-BDC': 'NC1=C(C=CC(=C1)C(=O)[O-])C(=O)[O-]',
    'BTB': 'c1cc(ccc1c2cc(cc(c2)c3ccc(cc3)C(=O)[O-])c4ccc(cc4)C(=O)[O-])C(=O)[O-]'
}

# --- ИСПРАВЛЕННАЯ ФУНКЦИЯ ---
def analyze_ligand(ligand_name):
    """
    Анализирует молекулу лиганда по его названию, извлекая различные химические дескрипторы.
    """
    smiles = ligand_smiles.get(ligand_name)
    if not smiles:
        return f"Лиганд {ligand_name} не найден в словаре."

    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return "Неверная строка SMILES."

    # --- Улучшенный подсчет подструктур ---
    # Паттерн для карбоксилатной группы
    carboxylate_pattern = Chem.MolFromSmarts('C(=O)[O-]')
    # Паттерн для карбоксильной кислоты (на всякий случай, если будут протонированные формы)
    carboxylic_acid_pattern = Chem.MolFromSmarts('C(=O)O')
    # Паттерн для первичной аминогруппы
    primary_amino_pattern = Chem.MolFromSmarts('[NH2]')
    
    # Подсчет количества совпадений для каждого паттерна
    num_carboxylate = len(mol.GetSubstructMatches(carboxylate_pattern))
    num_carboxylic_acid = len(mol.GetSubstructMatches(carboxylic_acid_pattern))
    num_amino_groups = len(mol.GetSubstructMatches(primary_amino_pattern))

    # Общее число карбоксильных групп (включая ионизированные)
    carboxyl_groups = num_carboxylate + num_carboxylic_acid
    
    # --- Правильный подсчет ароматических колец ---
    aromatic_rings = Lipinski.NumAromaticRings(mol)

    # Подсчет атомов по элементам
    carbon_atoms = sum(1 for atom in mol.GetAtoms() if atom.GetSymbol() == 'C')
    oxygen_atoms = sum(1 for atom in mol.GetAtoms() if atom.GetSymbol() == 'O')
    nitrogen_atoms = sum(1 for atom in mol.GetAtoms() if atom.GetSymbol() == 'N')

    # Вычисление стандартных дескрипторов
    molecular_weight = Descriptors.MolWt(mol)
    logP = Descriptors.MolLogP(mol)
    TPSA = Descriptors.TPSA(mol)
    h_bond_acceptors = Lipinski.NumHAcceptors(mol)
    h_bond_donors = Lipinski.NumHDonors(mol)

    # Возвращаем словарь со всеми вычисленными свойствами
    return {
        'carboxyl_groups (ligand)': carboxyl_groups,
        'aromatic_rings (ligand)': aromatic_rings,
        'amino_groups (ligand)': num_amino_groups, # Теперь это отдельный, более точный подсчет
        'carbon_atoms (ligand)': carbon_atoms,
        'oxygen_atoms (ligand)': oxygen_atoms,
        'nitrogen_atoms (ligand)': nitrogen_atoms, # Это общее число атомов N
        'molecular_weight (ligand)': molecular_weight,
        'logP (ligand)': logP,
        'TPSA (ligand)': TPSA,
        'h_bond_acceptors (ligand)': h_bond_acceptors,
        'h_bond_donors (ligand)': h_bond_donors
    }

def safe_generate_features(ligand_type):
    """
    Безопасная обертка для функции анализа, обрабатывающая возможные ошибки.
    """
    try:
        # Убедимся, что результат - это словарь (на случай возврата строки с ошибкой)
        features = analyze_ligand(ligand_type)
        if isinstance(features, dict):
            return features
        else:
            print(f"Info for ligand {ligand_type}: {features}")
            return {column: None for column in new_columns}
    except Exception as e:
        print(f"Critical error processing ligand {ligand_type}: {e}")
        return {column: None for column in new_columns}

# Определяем список новых столбцов
new_columns = [
    'carboxyl_groups (ligand)', 'aromatic_rings (ligand)', 'amino_groups (ligand)',
    'carbon_atoms (ligand)', 'oxygen_atoms (ligand)', 'nitrogen_atoms (ligand)',
    'molecular_weight (ligand)', 'logP (ligand)', 'TPSA (ligand)',
    'h_bond_acceptors (ligand)', 'h_bond_donors (ligand)'
]

# Включаем интеграцию tqdm с pandas
tqdm.pandas(desc="Processing Ligands")

# Применяем функцию к столбцу DataFrame
df['features'] = df['Лиганд'].progress_apply(safe_generate_features)

# "Разворачиваем" словарь с признаками в отдельные столбцы
for column in new_columns:
    df[column] = df['features'].apply(lambda x: x.get(column) if isinstance(x, dict) else None)

# Удаляем временный столбец
df = df.drop('features', axis=1)

### Добавление молярок, кол-ва вещества, соотношений

In [ ]:
metals_to_remove = ["Ti", "Ni", "Ce"]
df = df[~df["Металл"].isin(metals_to_remove)].reset_index(drop=True)

# --- Словари с молярными массами ---
# Примечание: это молярные массы СОЛЕЙ металлов (прекурсоров), а не чистых металлов.
metal_salt_molar_masses = {
    'Cu': 242, 
    'Zn': 297,
    'Al': 375,
    'Fe': 404,
    'Zr': 233,
    'Mg': 256,
    'La': 433,
    # 'Ce': 434, # Можно закомментировать или удалить, так как эти строки отфильтрованы
    'Y': 383
}

# Молярные массы лигандов (кислот)
ligand_acid_molar_masses = {
    'BTC': 207,
    'BDC': 164,
    'NH2-BDC': 179,
    'BTB': 435
}

# --- Расчеты новых признаков (остаются без изменений, т.к. написаны корректно) ---

# Создание столбцов с молярными массами с помощью .map()
df["Молярка_соли"] = df['Металл'].map(metal_salt_molar_masses)
df["Молярка_кислоты"] = df['Лиганд'].map(ligand_acid_molar_masses)

# Расчет количества вещества (молей)
df["n_соли"] = df["m (соли), г"] / df["Молярка_соли"]
df["n_кислоты"] = df["m(кис-ты), г"] / df["Молярка_кислоты"]

# Расчет мольного соотношения
# Добавляем небольшое число (эпсилон) в знаменатель для избежания деления на ноль, если это возможно
epsilon = 1e-9
df["n_ratio"] = df["n_соли"] / (df["n_кислоты"] + epsilon)

# Расчет соотношения объема растворителя к массе соли (мл/г)
df["Vsyn_m"] = df["Vсин. (р-ля), мл"] / (df["m (соли), г"] + epsilon)

## box noraml plots 

In [ ]:
import pandas as pd
import numpy as np
import pymatgen.core as mg
from rdkit import Chem
from rdkit.Chem import Descriptors

# --- 1. Генерация признаков для МЕТАЛЛОВ с помощью pymatgen ---
def get_metal_properties(metal_symbol):
    try:
        composition = mg.Composition(metal_symbol)
        element = composition.elements[0]
        return {
            'Total molecular weight (metal)': composition.weight,
            'Average ionic radius (metal)': element.average_ionic_radius,
            'Average electronegativity (metal)': composition.average_electroneg
        }
    except Exception as e:
        print(f"Не удалось обработать металл '{metal_symbol}': {e}")
        return {k: None for k in ['Total molecular weight (metal)', 'Average ionic radius (metal)', 'Average electronegativity (metal)']}

metal_features = df['Металл'].apply(get_metal_properties).apply(pd.Series)
df = pd.concat([df, metal_features], axis=1)


# --- 2. Генерация признаков для РАСТВОРИТЕЛЕЙ с помощью RDKit ---
solvent_smiles_russian = {
    'ДМФА': 'O=CN(C)C', 'Этанол': 'CCO', 'Вода': 'O', 'ДМСО': 'CS(=O)C', 'Ацетонитрил': 'CC#N'
}

def compute_solvent_descriptors(solvent_str):
    if not isinstance(solvent_str, str): return None
    components = [comp.strip() for comp in solvent_str.split('/')]
    smiles_list = [solvent_smiles_russian.get(comp) for comp in components if comp in solvent_smiles_russian]
    if not smiles_list: return None
    
    descriptors_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            descriptors_list.append({
                'Solvent_MolWt': Descriptors.MolWt(mol), 'Solvent_LogP': Descriptors.MolLogP(mol),
                'Solvent_NumHDonors': Descriptors.NumHDonors(mol), 'Solvent_NumHAcceptors': Descriptors.NumHAcceptors(mol)
            })
    if descriptors_list:
        return {key: np.mean([d[key] for d in descriptors_list]) for key in descriptors_list[0]}
    return None

solvent_features = df['Растворитель'].apply(compute_solvent_descriptors).apply(pd.Series)
df = pd.concat([df, solvent_features], axis=1)


# --- 3. Генерация ФИЗИКО-ХИМИЧЕСКИХ признаков ---
# Теперь этот код будет работать, так как названия столбцов стандартизированы
epsilon = 1e-9

# Используем стандартизированные названия столбцов (с одним пробелом)
df['Adsorption_Potential'] = df['E, кДж/моль'] * df['Ws, см3/г']
df['Capacity_Density'] = df['а0, ммоль/г'] / (df['SБЭТ, м2/г'] + epsilon)
df['SurfaceArea_MicroVol_Ratio'] = df['SБЭТ, м2/г'] / (df['W0, см3/г'] + epsilon)
df['Adsorption_Energy_Ratio'] = df['E, кДж/моль'] / (df['E0, кДж/моль'] + epsilon)
df['S_BET_E'] = df['SБЭТ, м2/г'] * df['E, кДж/моль']
df['x0_W0'] = df['х0, нм'] * df['W0, см3/г']

R = 8.314
T = 298.15
R_kJ = R / 1000

try:
    df['K_equilibrium'] = np.exp(df['E, кДж/моль'] / (R_kJ * T))
    df['Delta_G'] = -R_kJ * T * np.log(df['K_equilibrium'] + epsilon)
except (OverflowError, ValueError):
    print("Ошибка при расчете K_equilibrium или Delta_G.")
    df['K_equilibrium'], df['Delta_G'] = np.nan, np.nan

E_J_per_mol = df['E, кДж/моль'] * 1000
df["B_micropore"] = np.power(((2.303 * R) / (E_J_per_mol + epsilon)), 2)

### Посмотрим распределение признаков

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- 1. One-Hot Encoding (ваш код, он корректен) ---
df_encoded = df.copy()
categorical_vars = ['Металл', 'Лиганд', 'Растворитель']
df_encoded = pd.get_dummies(df_encoded, columns=categorical_vars, prefix=categorical_vars, drop_first=False, dtype=int)


# --- 2. Автоматический и надежный выбор признаков для корреляции ---
# Выбираем все столбцы с числовыми данными
numerical_features = df_encoded.select_dtypes(include=np.number).columns.tolist()

# Удаляем идентификаторы или столбцы, которые не должны участвовать в корреляции
# (если они числовые)
columns_to_exclude = ['№ Образца']
features_for_corr = [col for col in numerical_features if col not in columns_to_exclude]


# --- 3. Расчет корреляционной матрицы ---
# Теперь мы используем программно созданный список 'features_for_corr'
corr_matrix = df_encoded[features_for_corr].corr()


# --- 4. Визуализация ---

# --- График 1: Общая тепловая карта (без аннотаций для читаемости) ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(28, 22))
heatmap = sns.heatmap(
    corr_matrix,
    cmap='coolwarm', # Контрастная палитра
    vmin=-1, vmax=1   # Фиксируем диапазон цвета от -1 до 1
)
heatmap.set_title('Тепловая карта корреляций всех признаков', fontdict={'fontsize':20}, pad=12)
plt.show()


# --- График 2: Более полезный график корреляций с целевой переменной ---
# Выберем целевую переменную, например, адсорбционную емкость
# target_variable = 'а0, ммоль/г'

# if target_variable in corr_matrix:
#     # Получаем столбец корреляций для нашей цели
#     corr_target = corr_matrix[target_variable].drop(target_variable)
    
#     # Сортируем значения для наглядности
#     corr_target_sorted = corr_target.sort_values(ascending=False)

#     # Строим столбчатую диаграмму
#     plt.figure(figsize=(12, 16))
#     sns.barplot(x=corr_target_sorted.values, y=corr_target_sorted.index, palette='vlag')
#     plt.title(f'Корреляция признаков с целевой переменной: "{target_variable}"', fontsize=16)
#     plt.xlabel('Коэффициент корреляции Пирсона', fontsize=12)
#     plt.ylabel('Признаки', fontsize=12)
#     plt.grid(axis='x', linestyle='--', alpha=0.7)
#     plt.tight_layout()
#     plt.show()
# else:
#     print(f"Целевая переменная '{target_variable}' не найдена в матрице корреляций.")

In [ ]:
# --- Код для экспорта ---
output_filename = 'data/SEC_SYN_with_features.csv'

try:
    # Сохраняем DataFrame в CSV файл
    # index=False - чтобы не записывать индекс DataFrame как отдельный столбец
    # encoding='utf-8-sig' - для правильного отображения кириллицы в Excel
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    
    print(f"DataFrame успешно сохранен в файл: '{output_filename}'")
    
except Exception as e:
    print(f"Произошла ошибка при сохранении файла: {e}")